# Rollback Predictor Classifier 

In [2]:
# importing all the libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import codecs  
from tqdm import tqdm
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize import word_tokenize
import os, codecs
from keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle


from tensorflow.keras.preprocessing import  sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
plt.style.use('fivethirtyeight')
print(tf.__version__)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(filename)

2.3.0


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ranji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ranji\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Fetching the dataset

### Stack Overflow Dataset was obtained from the Stack Exchange using queries


In [3]:
# change the directory of the csv files

# Contains reverted post body
df_reverted = pd.read_csv('C:/Users/ranji/Desktop/SE/Project/df_reverted_body.csv')

#Contains the list of modifications made to the post body
df_modified = pd.read_csv('C:/Users/ranji/Desktop/SE/Project/df_phCombined.csv')

In [4]:
# Getting the Non roll back and rollbacked post bodies using the Post's PostId and UserId
new_df = pd.merge(df_reverted, df_modified,  how='left', left_on=['PostId','UserId'], right_on = ['PostId','UserId'])

In [9]:
new_df.head()

,Unnamed: 0_x,Id_x,PostHistoryTypeId_x,PostId,RevisionGUID_x,CreationDate_x,UserId,UserDisplayName_x,Comment_x,Text_x,ContentLicense_x,Unnamed: 0_y,Id_y,PostHistoryTypeId_y,RevisionGUID_y,CreationDate_y,UserDisplayName_y,Comment_y,Text_y,ContentLicense_y
0,0,163783933,8,48045396,e7e390f3-b1f6-4cdd-9367-88b5cd542d35,2018-01-01 00:53:19,5959819.0,NaN,Rollback to [657ce1d1-80ca-4229-ad48-3f66981ac...,What is the difference of an implicitly define...,CC BY-SA 3.0,11388.0,163783928.0,5.0,e7e2433d-33e4-47e7-b38a-da3aea63310e,2018-01-01 00:52:38,NaN,added 2 characters in body,What is the difference of an implicitly define...,CC BY-SA 3.0
1,1,163786119,8,48044278,341ee45d-5320-4ec8-9dbd-870e0e436ea5,2018-01-01 03:36:43,1136195.0,NaN,Rollback to [4b7b6472-3fcf-4278-940b-8105f868d...,"The following code is working, but I don't wan...",CC BY-SA 3.0,10975.0,163775292.0,5.0,aaf17bb4-e58d-4069-ac4f-5b6fd1b69b42,2017-12-31 17:25:55,NaN,deleted 71 characters in body,"The following code is working, but I don't wan...",CC BY-SA 3.0
2,1,163786119,8,48044278,341ee45d-5320-4ec8-9dbd-870e0e436ea5,2018-01-01 03:36:43,1136195.0,NaN,Rollback to [4b7b6472-3fcf-4278-940b-8105f868d...,"The following code is working, but I don't wan...",CC BY-SA 3.0,10976.0,163775319.0,5.0,4b7b6472-3fcf-4278-940b-8105f868dcfd,2017-12-31 17:27:31,NaN,deleted 3 characters in body,"The following code is working, but I don't wan...",CC BY-SA 3.0
3,2,163789315,8,48047744,a013402b-e6b1-48b3-92e2-b88b0017437f,2018-01-01 06:58:52,5539082.0,NaN,Rollback to [abe84ec8-fdaf-407c-8256-67def83cc...,> ViewPager not refreshing in android\r\n\r\n*...,CC BY-SA 3.0,10992.0,163789415.0,5.0,dd1f84f5-6850-4218-93ee-857997658d0a,2018-01-01 07:04:10,NaN,added 53 characters in body,> ViewPager not refreshing in android\r\n\r\n*...,CC BY-SA 3.0
4,2,163789315,8,48047744,a013402b-e6b1-48b3-92e2-b88b0017437f,2018-01-01 06:58:52,5539082.0,NaN,Rollback to [abe84ec8-fdaf-407c-8256-67def83cc...,> ViewPager not refreshing in android\r\n\r\n*...,CC BY-SA 3.0,10994.0,163789492.0,5.0,698c7170-454c-45ee-be77-b24fb309c1e4,2018-01-01 07:08:48,NaN,added 93 characters in body,>if i change the image in server(different ima...,CC BY-SA 3.0


In [10]:
# non reverted post bodies
class0 = pd.DataFrame()
class0["data"] = new_df["Text_x"]
class0["labels"] = 0
class0.head()

,data,labels
0,What is the difference of an implicitly define...,0
1,"The following code is working, but I don't wan...",0
2,"The following code is working, but I don't wan...",0
3,> ViewPager not refreshing in android\r\n\r\n*...,0
4,> ViewPager not refreshing in android\r\n\r\n*...,0


In [12]:
# reverted post bodies
class1 = pd.DataFrame()
class1["data"] = new_df["Text_y"]
class1["labels"] = 1
class1.head()

,data,labels
0,What is the difference of an implicitly define...,1
1,"The following code is working, but I don't wan...",1
2,"The following code is working, but I don't wan...",1
3,> ViewPager not refreshing in android\r\n\r\n*...,1
4,>if i change the image in server(different ima...,1


In [13]:
# Combining both classes
df_dataset = pd.concat([class0, class1], axis=0)

In [14]:
# Shuffling the dataset
dataset = shuffle(df_dataset)

In [16]:
# Splitting the dataset for training and testing the classifier
X_train, X_test, y_train, y_test = train_test_split(dataset["data"], dataset["labels"], test_size=0.2, random_state=42)

## Data Preprocessing and Vectorization

In [19]:
dataset['doc_len'] = dataset['data'].apply(lambda words: len(str(words).split(" ")))
max_seq_len = np.round(dataset['doc_len'].mean() + dataset['doc_len'].std()).astype(int)

In [20]:
max_features = 20000
max_text_length = 400

In [21]:
# Fetching the Glove word Emembeddings
print('loading word embeddings...')

embeddings_index = {}
f = codecs.open('C:/Users/ranji/Desktop/NLP/Project/twitter hate speech/Code/Dataset/glove.840B.300d/glove.840B.300d.txt', encoding='utf-8')

for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('found %s word vectors' % len(embeddings_index))

695it [00:00, 6930.92it/s]

loading word embeddings...


2196018it [05:24, 6777.13it/s]

found 2196016 word vectors


In [22]:
MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))


In [23]:
raw_docs_train = X_train.to_list()
raw_docs_test = X_test.to_list()


print("pre-processing train data...")

processed_docs_train = []
for doc in tqdm(raw_docs_train):
    tokens = word_tokenize(str(doc))
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_train.append(" ".join(filtered))
#end for

processed_docs_test = []
for doc in tqdm(raw_docs_test):
    tokens = word_tokenize(str(doc))
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))
#end for

print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

  0%|          | 50/146795 [00:00<05:00, 488.82it/s]

pre-processing train data...


100%|██████████| 36699/36699 [01:20<00:00, 458.37it/s]


tokenizing input data...
dictionary size:  254068


In [ ]:
#Embedding matrix parameters
MAX_NB_WORDS = 100000
num_filters = 64 
embed_dim = 300 
weight_decay = 1e-4

## Generating an glove embedding matrix for the LSTM classifier

In [157]:
#embedding matrix

print('preparing embedding matrix...')

words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))

for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 63906


## LSTM model structure

In [164]:
from keras.layers import BatchNormalization
import tensorflow as tf

model = tf.keras.Sequential()

model.add(Embedding(nb_words,embed_dim,input_length=max_seq_len, weights=[embedding_matrix],trainable=False))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 973, 300)          30000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                85248     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 30,087,361
Trainable params: 87,361
Non-trainable params: 30,000,000
_________________________________________________________________


In [165]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Training the LSTM model

### To save time, load the attached trained rollBackPredictor model instead of training it from the scratch

In [402]:
es_callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(word_seq_train, y_train, batch_size= 512, epochs = 25, validation_split=0.2, callbacks=[es_callback])
model.save('rollBackPredictor.h5')

Epoch 1/2
230/230 [==============================] - 15290s 66s/step - loss: 0.5655 - accuracy: 0.6671 - val_loss: 0.5645 - val_accuracy: 0.6656
Epoch 2/2
230/230 [==============================] - 16205s 70s/step - loss: 0.5612 - accuracy: 0.6709 - val_loss: 0.5627 - val_accuracy: 0.6665


### To save time, load the attached trained rollBackPredictor model instead of training it from the scratch

# Model Evaluation

In [ ]:
model = load_model('rollBackPredictor.h5')

In [289]:
y_pred = model.predict(word_seq_test,verbose=1,batch_size=32)

1147/1147 [==============================] - 286s 250ms/step


In [291]:
final_y_pred = [0 if y[0] < 0.5 else 1 for y in y_pred]

In [292]:
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, final_y_pred) * 100))

Accuracy: 66.41%


In [24]:
y_pred = model.predict(word_seq_test,verbose=1,batch_size=32)

1147/1147 [==============================] - 188s 164ms/step


In [25]:
final_y_pred = [0 if y[0] < 0.5 else 1 for y in y_pred]
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, final_y_pred) * 100))


Accuracy: 66.11%


In [26]:
print("precision_score: {:.6f}".format(precision_score(y_test, final_y_pred, average='macro')))

precision_score: 0.773493


In [27]:
print("recall_score: {:.6f}".format(recall_score(y_test, final_y_pred, average='macro')))

recall_score: 0.663000


In [28]:
print("F1 Score: {:.6f}".format(f1_score(y_test, final_y_pred, average='macro')))

F1 Score: 0.623798
